In [1]:
using ProgressMeter
using Random #, Distributions
using PyPlot
using CUDA
using StaticArrays
D = 2;

In [2]:
mutable struct PTParams{T}
    TMax       :: T
    λ          :: T
    NTemps     :: Int
    Nexchanges :: Int
    PTParams{T}() where {T} = new()
end;

mutable struct MetropolisParams{T}
    NSteps       :: Int
    StepSize     :: T
    GaussianStep :: Bool
    MetropolisParams{T}() where {T} = new()
end;

# data structure for the points of the phase space

x is a tensor with NTemp walkers, each one with d dimensions. The structure will be:

$x[temp, dim]$

In [3]:
function energy(x::Vector{T}, λ, b, W, c) where {T}
    # Example energy function: sum of squares
    return sum(x .^ 2) # TODO: replace with actual energy function of the physical system
end

energy (generic function with 1 method)

In [4]:
function energy_CUDA(x, λ, b, W, c, energies)
    i = (blockIdx().x - 1) * blockDim().x + threadIdx().x
    Nchains = size(x, 1)
    D = size(x, 2)
    if i <= Nchains
        energies[i] = energy(x[i, :], λ, b, W, c)
    end
    return
end

energy_CUDA (generic function with 1 method)

In [ ]:
function gauss_2d(x ,μ, σ)
    return exp(-0.5 * ((x[1] - μ[1])^2 + (x[2] - μ[2])^2) / σ^2)
end

@inline function gauss_nd(x::NTuple{D, T}, μ::NTuple{D, T}, σ::T) where {D, T}
    s = zero(T)
    @inbounds for i in 1:D
        s += (x[i] - μ[i])^2
    end
    return exp(-0.5 * s / σ^2)
end


@inline function ener_landscape(x::NTuple{D, T}) where {D, T}
    # μ1 = @SVector zeros(T, D)
    # μ2 = @SVector fill(T(5.0), D)
    return gauss_nd(x, 0.0, 1.0) + 0.5 * gauss_nd(x, 5.0, 1.0)
end

function mock_energy_CUDA(x, energies)
    i = (blockIdx().x - 1) * blockDim().x + threadIdx().x
    Nchains = size(x, 1)
    if i <= Nchains

        # Inlining the Gaussian landscape calculation directly into the kernel
        μ1 = zero(Float64)  # μ1 = (0,0) (zero vector)
        μ2 = 5.0  # μ2 = (5,5) (scalar for simplicity)
        
        s1 = zero(Float64)
        s2 = zero(Float64)
        
        # Compute squared differences directly in the kernel
        @inbounds for j in 1:D
            s1 += (x[i,j] - μ1)^2
            s2 += (x[i,j] - μ2)^2
        end
        
        energy = exp(-0.5 * s1) + 0.5 * exp(-0.5 * s2)  # Combine the Gaussian terms
        
        energies[i] = energy



        # energies[i] = ener_landscape(x_actual)
    end
    return
end

mock_energy_CUDA (generic function with 1 method)

In [6]:
x = range(-10, stop=10, length=100)
y = range(-10, stop=10, length=100)
# generate a matrix of x and y values, without meshgrid
X = zeros(length(x), length(y))
Y = zeros(length(x), length(y))
for i in 1:length(x)
    for j in 1:length(y)
        X[i, j] = x[i]
        Y[i, j] = y[j]
    end
end

Z = zeros(size(X))
for i in 1:size(X, 1)
    for j in 1:size(X, 2)
        Z[i, j] = ener_landscape([X[i, j], Y[i, j]])
    end
end
fig = figure()
ax = fig.add_subplot(111)

ax.contourf(X, Y, Z, levels=50)
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_title("Energy Landscape")
savefig("energy_landscape.png")

MethodError: MethodError: no method matching ener_landscape(::Vector{Float64})
The function `ener_landscape` exists, but no method is defined for this combination of argument types.

Closest candidates are:
  ener_landscape(!Matched::NTuple{D, T}) where {D, T}
   @ Main ~/PhD/1D-Variational-RBM/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W5sZmlsZQ==.jl:14


In [7]:
function parallel_tempering(PTParameters::PTParams, MetropolisParams::MetropolisParams, initial_guess)

    # -------------------------------------------------------
    # Local variables
    # -------------------------------------------------------
    
    NTemps           = PTParameters.NTemps
    Nexchanges       = PTParameters.Nexchanges
    TMax             = PTParameters.TMax
    NSteps           = MetropolisParams.NSteps
    StepSize         = MetropolisParams.StepSize

    λ                = PTParameters.λ
    temperatures     = zeros(NTemps)
    temperatures[1]  = TMax
    [temperatures[i] = temperatures[i-1]*λ for i in 2:NTemps]

    xo               = initial_guess
    xn               = zeros(size(xo))
    D                = size(xo, 2)

    Eveco           = zeros(NTemps) # preallocate energy vector
    @cuda mock_energy_CUDA(xo, Eveco) # compute parallel energy
    TupEBest        = findmin(Eveco)
    EBest           = TupEBest[1]
    EBestPos        = xo[TupEBest[2]]

    display("Initial guess: ", xo)
    display("Initial energies: ", Eveco)
    display("Initial best energy: ", EBest[1])
    display("Temperatures: ", temperatures')
    # debug energy and init guess
    is_plot = false
    if is_plot
        clf()
        plot(-10:10, Ener.(-10:10), label="Energy")
        plot(xo, Ener.(xo), "ro", label="Initial guess")
        plot(EBestPos, EBest[1], "go", label="Best guess")
        title("Initial guess")
        xlabel("x")
        ylabel("Energy")
        legend()
        # if ./out exists, save the figure there
        if isdir("./out")
            savefig("./out/energy")
        else
            show()
        end
    end

    # -------------------------------------------------------
    # Parallel tempering loop
    # -------------------------------------------------------
    @showprogress for _ in 1:PTParameters.Nexchanges+1
        # Metropolis step
        for i in 1:NSteps
            xn = xo .+ StepSize*randn(NTemps, D) # generate new positions
            Evecn = zeros(NTemps) # preallocate energy vector
            @cuda threads=256 blocks=256 mock_energy_CUDA(xn, Evecn) # compute parallel energy
            ΔE_vec = Evecn .- Eveco # compute the energy difference between the new and old positions

            mask1 = ΔE_vec .< 0 # if the new position is better, accept it
            mask2 = !mask1 & (rand(NTemps) .< exp.(-ΔE_vec./temperatures)) # if the new position is worse, metropolis probability
            mask = mask1 .| mask2 # combine the two masks
            for i in 1:D
                xo[i] = xn[i] .* mask .+ xo[i] .* .!mask
            end

            Eveco = zeros(NTemps) # preallocate energy vector
            @cuda threads=256 blocks=256 mock_energy_CUDA(xo, Eveco) # compute parallel energy
            prob_best_guess = findmin(Eveco) # from the new sampled energies, find the best guess
            if prob_best_guess[1] < EBest # compare the new best guess with the old one. If its better, update it
                # EBest: energy, position
                # EBestPos: position
                EBest    = prob_best_guess[1]
                EBestPos = xo[prob_best_guess[2]]
            end
        end

        # Exchange step
        # no check for the lowest state, as we don't explore space here
        exchange_energies = zeros(NTemps)
        @cuda threads=256 blocks=256 mock_energy_CUDA(xo, exchange_energies) # compute parallel energy
        for temp in 1:NTemps-1
            ΔE_exchange_no_T = exchange_energies[temp] - exchange_energies[temp+1]
            ΔE_exchange = ΔE_exchange_no_T * (1/temperatures[temp] - 1/temperatures[temp+1])
            if ΔE_exchange < 0
                xo[temp, :], xo[temp+1, :] = xo[temp+1, :], xo[temp, :]
            elseif rand() < exp(-ΔE_exchange)
                xo[temp, :], xo[temp+1, :] = xo[temp+1, :], xo[temp, :]
            end
        end
    end

    return xo, EBest, EBestPos
end;

In [14]:
NTemps = 10
test = zeros(NTemps)
x_tests = randn(NTemps, 2) # random initial guess
test = CuArray(test)
x_tests = CuArray(x_tests)
@cuda threads=256 blocks=256 mock_energy_CUDA(x_tests, test)
# check if the energies are computed correctly
for energy in test
    println(energy)
end

ErrorException: Scalar indexing is disallowed.
Invocation of getindex resulted in scalar indexing of a GPU array.
This is typically caused by calling an iterating implementation of a method.
Such implementations *do not* execute on the GPU, but very slowly on the CPU,
and therefore should be avoided.

If you want to allow scalar iteration, use `allowscalar` or `@allowscalar`
to enable scalar iteration globally or for the operations in question.